# Explore the PEP extraction results on the Empkins data

Observations:  
- For each phase the heartbeat ID starts at zero

## Setup and helper functions

In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.datasets import EmpkinsDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../")

In [3]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

\Users\sebas\Development\ResearchInternship\Data\2024_08_PEP_Benchmarking\EmpkinS_Dataset


#### Specify whether the results should be saved or not

In [4]:
save_results = True

In [5]:
result_path = root_path.joinpath("results")
result_path

WindowsPath('../../results')

In [6]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
fs_empkins = dataset_empkins.sampling_rate_icg
print(f"Sampling rate ICG: {fs_empkins}")
dataset_empkins

Sampling rate ICG: 1000


EmpkinsDataset [150 groups/rows]

       participant condition    phase
   0        VP_001      tsst     Prep
   1        VP_001      tsst  Pause_1
   2        VP_001      tsst     Talk
   3        VP_001      tsst     Math
   4        VP_001      tsst  Pause_5
   ..          ...       ...      ...
   145      VP_032     ftsst     Prep
   146      VP_032     ftsst  Pause_1
   147      VP_032     ftsst     Talk
   148      VP_032     ftsst     Math
   149      VP_032     ftsst  Pause_5
   
   [150 rows x 3 columns]

In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

In [8]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_q_wave").resolve(), index_cols_per_sample=["participant", "condition", "phase"]
)

In [9]:
results_empkins.per_sample

heartbeat_id  \
                                                                                                                  estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                     
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep    0             0   
                                                                                                            1             1   
                                                                                                            2             2   
                                                                                                            3             3   
                                                                                                            4             4   
...                                                                                                                     ...   
                 stern1985                       none                         VP_032      ftsst     Pause_5 7             7   
                                                                                                            8             8   
                                                                                                            9             9   
                                                                                                            10           10   
                                                                                                            11           11   

                                                                                                                          \
                                                                                                               reference   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                  
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep    0          0   
                                                                                                            1          1   
                                                                                                            2          2   
                                                                                                            3          3   
                                                                                                            4          4   
...                                                                                                                  ...   
                 stern1985                       none                         VP_032      ftsst     Pause_5 7          7   
                                                                                                            8          8   
                                                                                                            9          9   
                                                                                                            10        10   
                                                                                                            11        11   

                                                                                                               heartbeat_start_sample  \
                                                                                                                            estimated   
q_wave_algorithm b_point_algorithm               outlier_correction_algorithm participant condition phase                               
q-wave-reference arbol2017-isoelectric-crossings forouzanfar2018              VP_001      tsst      Prep    0                     394   
                                               

In [10]:
results_empkins_b_point_estimated = results_empkins.per_sample.droplevel("q_wave_algorithm")[[("heartbeat_id", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference")]]
results_empkins_b_point_estimated = results_empkins_b_point_estimated.reset_index()
results_empkins_b_point_estimated

b_point_algorithm outlier_correction_algorithm  \
                                                                       
0       arbol2017-isoelectric-crossings              forouzanfar2018   
1       arbol2017-isoelectric-crossings              forouzanfar2018   
2       arbol2017-isoelectric-crossings              forouzanfar2018   
3       arbol2017-isoelectric-crossings              forouzanfar2018   
4       arbol2017-isoelectric-crossings              forouzanfar2018   
...                                 ...                          ...   
149995                        stern1985                         none   
149996                        stern1985                         none   
149997                        stern1985                         none   
149998                        stern1985                         none   
149999                        stern1985                         none   

       participant condition    phase level_5 heartbeat_id b_point_sample  \
                                                 reference      estimated   
0           VP_001      tsst     Prep       0            0          669.0   
1           VP_001      tsst     Prep       1            1         1238.0   
2           VP_001      tsst     Prep       2            2         1834.0   
3           VP_001      tsst     Prep       3            3         2394.0   
4           VP_001      tsst     Prep       4            4         2943.0   
...            ...       ...      ...     ...          ...            ...   
149995      VP_032     ftsst  Pause_5       7            7         6210.0   
149996      VP_032     ftsst  Pause_5       8            8         7052.0   
149997      VP_032     ftsst  Pause_5       9            9         7857.0   
149998      VP_032     ftsst  Pause_5      10           10         8667.0   
149999      VP_032     ftsst  Pause_5      11           11         9463.0   

                  
       reference  
0            634  
1           1206  
2           1811  
3           2382  
4           2926  
...          ...  
149995      6256  
149996      7054  
149997      7855  
149998      8673  
149999      9467  

[150000 rows x 9 columns]

In [11]:
results_empkins_b_point_estimated.columns = results_empkins_b_point_estimated.columns.to_flat_index().str.join("")
results_empkins_b_point_estimated = results_empkins_b_point_estimated.drop(columns=["level_5"])
results_empkins_b_point_estimated

b_point_algorithm outlier_correction_algorithm  \
0       arbol2017-isoelectric-crossings              forouzanfar2018   
1       arbol2017-isoelectric-crossings              forouzanfar2018   
2       arbol2017-isoelectric-crossings              forouzanfar2018   
3       arbol2017-isoelectric-crossings              forouzanfar2018   
4       arbol2017-isoelectric-crossings              forouzanfar2018   
...                                 ...                          ...   
149995                        stern1985                         none   
149996                        stern1985                         none   
149997                        stern1985                         none   
149998                        stern1985                         none   
149999                        stern1985                         none   

       participant condition    phase  heartbeat_idreference  \
0           VP_001      tsst     Prep                      0   
1           VP_001      tsst     Prep                      1   
2           VP_001      tsst     Prep                      2   
3           VP_001      tsst     Prep                      3   
4           VP_001      tsst     Prep                      4   
...            ...       ...      ...                    ...   
149995      VP_032     ftsst  Pause_5                      7   
149996      VP_032     ftsst  Pause_5                      8   
149997      VP_032     ftsst  Pause_5                      9   
149998      VP_032     ftsst  Pause_5                     10   
149999      VP_032     ftsst  Pause_5                     11   

        b_point_sampleestimated  b_point_samplereference  
0                         669.0                      634  
1                        1238.0                     1206  
2                        1834.0                     1811  
3                        2394.0                     2382  
4                        2943.0                     2926  
...                         ...                      ...  
149995                   6210.0                     6256  
149996                   7052.0                     7054  
149997                   7857.0                     7855  
149998                   8667.0                     8673  
149999                   9463.0                     9467  

[150000 rows x 8 columns]

In [12]:
results_empkins_b_point_estimated["b_point_algorithm_combi"] = results_empkins_b_point_estimated["b_point_algorithm"] + "_" + results_empkins_b_point_estimated["outlier_correction_algorithm"]
results_empkins_b_point_estimated = results_empkins_b_point_estimated.drop(columns=["b_point_algorithm", "outlier_correction_algorithm"])
results_empkins_b_point_estimated.reindex(level=["particpant", "condition", "phase"])

participant condition    phase  heartbeat_idreference  \
0           VP_001      tsst     Prep                      0   
1           VP_001      tsst     Prep                      1   
2           VP_001      tsst     Prep                      2   
3           VP_001      tsst     Prep                      3   
4           VP_001      tsst     Prep                      4   
...            ...       ...      ...                    ...   
149995      VP_032     ftsst  Pause_5                      7   
149996      VP_032     ftsst  Pause_5                      8   
149997      VP_032     ftsst  Pause_5                      9   
149998      VP_032     ftsst  Pause_5                     10   
149999      VP_032     ftsst  Pause_5                     11   

        b_point_sampleestimated  b_point_samplereference  \
0                         669.0                      634   
1                        1238.0                     1206   
2                        1834.0                     1811   
3                        2394.0                     2382   
4                        2943.0                     2926   
...                         ...                      ...   
149995                   6210.0                     6256   
149996                   7052.0                     7054   
149997                   7857.0                     7855   
149998                   8667.0                     8673   
149999                   9463.0                     9467   

                                b_point_algorithm_combi  
0       arbol2017-isoelectric-crossings_forouzanfar2018  
1       arbol2017-isoelectric-crossings_forouzanfar2018  
2       arbol2017-isoelectric-crossings_forouzanfar2018  
3       arbol2017-isoelectric-crossings_forouzanfar2018  
4       arbol2017-isoelectric-crossings_forouzanfar2018  
...                                                 ...  
149995                                   stern1985_none  
149996                                   stern1985_none  
149997                                   stern1985_none  
149998                                   stern1985_none  
149999                                   stern1985_none  

[150000 rows x 7 columns]

In [13]:
print("Possible B-Point algorithm combinations: ")
algo_list = results_empkins_b_point_estimated["b_point_algorithm_combi"].unique()
print(f"There are {len(algo_list)} possible algorithm combinations:")
algo_list

Possible B-Point algorithm combinations: 
There are 30 possible algorithm combinations:


array(['arbol2017-isoelectric-crossings_forouzanfar2018',
       'arbol2017-isoelectric-crossings_linear-interpolation',
       'arbol2017-isoelectric-crossings_none',
       'arbol2017-second-derivative_forouzanfar2018',
       'arbol2017-second-derivative_linear-interpolation',
       'arbol2017-second-derivative_none',
       'arbol2017-third-derivative_forouzanfar2018',
       'arbol2017-third-derivative_linear-interpolation',
       'arbol2017-third-derivative_none',
       'debski1993-second-derivative_forouzanfar2018',
       'debski1993-second-derivative_linear-interpolation',
       'debski1993-second-derivative_none', 'drost2022_forouzanfar2018',
       'drost2022_linear-interpolation', 'drost2022_none',
       'forounzafar2018_forouzanfar2018',
       'forounzafar2018_linear-interpolation', 'forounzafar2018_none',
       'lozano2007-linear-regression_forouzanfar2018',
       'lozano2007-linear-regression_linear-interpolation',
       'lozano2007-linear-regression_none',
    

In [14]:
print(f"Number of rows in the long dataframe: {results_empkins_b_point_estimated.shape[0]}")
print(f"Number of possible B-Point algorithm combinations: {len(algo_list)}")
print(f"Expected amount of rows in the wide datframe: {results_empkins_b_point_estimated.shape[0]/len(algo_list)}")

Number of rows in the long dataframe: 150000
Number of possible B-Point algorithm combinations: 30
Expected amount of rows in the wide datframe: 5000.0


### Convert the dataframe from the long to the wide format

In [15]:
# Pivot the DataFrame
df_empkins_pivot = results_empkins_b_point_estimated.pivot_table(
    index=['participant', 'condition', 'phase', 'heartbeat_idreference', 'b_point_samplereference'],
    columns='b_point_algorithm_combi',
    values='b_point_sampleestimated'
).reset_index()

# Flatten the columns
df_empkins_pivot.columns.name = None
df_empkins_pivot.columns = [f'{col}' if isinstance(col, str) else f'{col[1]}' for col in df_empkins_pivot.columns]
df_empkins_pivot

participant condition phase  heartbeat_idreference  \
0         VP_001     ftsst  Math                      1   
1         VP_001     ftsst  Math                      2   
2         VP_001     ftsst  Math                      3   
3         VP_001     ftsst  Math                      4   
4         VP_001     ftsst  Math                      5   
...          ...       ...   ...                    ...   
4990      VP_032      tsst  Talk                     39   
4991      VP_032      tsst  Talk                     40   
4992      VP_032      tsst  Talk                     41   
4993      VP_032      tsst  Talk                     42   
4994      VP_032      tsst  Talk                     43   

      b_point_samplereference  \
0                        1074   
1                        1849   
2                        2518   
3                        3252   
4                        3933   
...                       ...   
4990                    26419   
4991                    27042   
4992                    27719   
4993                    28395   
4994                    29097   

      arbol2017-isoelectric-crossings_forouzanfar2018  \
0                                              1111.0   
1                                              1844.0   
2                                              2558.0   
3                                              3264.0   
4                                              3942.0   
...                                               ...   
4990                                          26444.0   
4991                                          27051.0   
4992                                          27696.0   
4993                                          28449.0   
4994                                          29132.0   

      arbol2017-isoelectric-crossings_linear-interpolation  \
0                                                1111.0      
1                                                1844.0      
2                                                2558.0      
3                                                3264.0      
4                                                3942.0      
...                                                 ...      
4990                                            26444.0      
4991                                            27051.0      
4992                                            27696.0      
4993                                            28449.0      
4994                                            29132.0      

      arbol2017-isoelectric-crossings_none  \
0                                   1111.0   
1                                   1844.0   
2                                   2558.0   
3                                   3264.0   
4                                   3942.0   
...                                    ...   
4990                               26444.0   
4991                               27051.0   
4992                               27696.0   
4993                               28449.0   
4994                               29132.0   

      arbol2017-second-derivative_forouzanfar2018  \
0                                          1080.0   
1                                          1810.0   
2                                          2494.0   
3                                          3215.0   
4                                          3881.0   
...                                           ...   
4990                                      26385.0   
4991                                      27018.0   
4992                                      27679.0   
4993                                      28408.0   
4994                                      29103.0   

      arbol2017-second-derivative_linear-interpolation  ...  \
0                                               1080.0  ...   
1                                               1809.0  ...   
2                                               2494.0  ...   
3                          

In [16]:
print(f"Max heartbeat id long dataframe: {max(results_empkins_b_point_estimated["heartbeat_idreference"])}")
print(f"Max heartbeat id wide dataframe: {max(df_empkins_pivot["heartbeat_idreference"])}")

Max heartbeat id long dataframe: 73
Max heartbeat id wide dataframe: 73


In [17]:
vp_001_tsst_long = results_empkins_b_point_estimated[results_empkins_b_point_estimated["participant"] == "VP_001"][results_empkins_b_point_estimated["condition"] == "tsst"]
vp_01_tsst_wide = df_empkins_pivot[df_empkins_pivot["participant"] == "VP_001"][df_empkins_pivot["condition"] == "tsst"]

C:\Users\sebas\AppData\Local\Temp\ipykernel_33960\4111976217.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vp_001_tsst_long = results_empkins_b_point_estimated[results_empkins_b_point_estimated["participant"] == "VP_001"][results_empkins_b_point_estimated["condition"] == "tsst"]
C:\Users\sebas\AppData\Local\Temp\ipykernel_33960\4111976217.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vp_01_tsst_wide = df_empkins_pivot[df_empkins_pivot["participant"] == "VP_001"][df_empkins_pivot["condition"] == "tsst"]


In [18]:
print(f"Lenght of VP01 - tsst long / Number of B-Point algorithm combinations: {vp_001_tsst_long.shape[0]/len(algo_list)} should match length of VP01 - tsst wide: {vp_01_tsst_wide.shape[0]}")

Lenght of VP01 - tsst long / Number of B-Point algorithm combinations: 179.0 should match length of VP01 - tsst wide: 179


### Save the pivot dataframe

In [19]:
if save_results:
    df_empkins_pivot.to_csv(result_path.joinpath("pivot_dataframe_b_point/empkins_pivot_dataframe_b_point_all_algos.csv"))

### Convert samples to ms

In [20]:
df_empkins_pivot_ms = df_empkins_pivot.copy()
exclude_cols = ["participant", "condition", "phase", "heartbeat_idreference"]
df_empkins_pivot_ms.loc[:, ~df_empkins_pivot_ms.columns.isin(exclude_cols)] = df_empkins_pivot_ms.loc[:, ~df_empkins_pivot_ms.columns.isin(exclude_cols)].apply(lambda x: x * convert_hz_to_ms(fs_empkins))
df_empkins_pivot_ms          

C:\Users\sebas\AppData\Local\Temp\ipykernel_33960\4067454933.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_empkins_pivot_ms.loc[:, ~df_empkins_pivot_ms.columns.isin(exclude_cols)] = df_empkins_pivot_ms.loc[:, ~df_empkins_pivot_ms.columns.isin(exclude_cols)].apply(lambda x: x * convert_hz_to_ms(fs_empkins))


participant condition phase  heartbeat_idreference  \
0         VP_001     ftsst  Math                      1   
1         VP_001     ftsst  Math                      2   
2         VP_001     ftsst  Math                      3   
3         VP_001     ftsst  Math                      4   
4         VP_001     ftsst  Math                      5   
...          ...       ...   ...                    ...   
4990      VP_032      tsst  Talk                     39   
4991      VP_032      tsst  Talk                     40   
4992      VP_032      tsst  Talk                     41   
4993      VP_032      tsst  Talk                     42   
4994      VP_032      tsst  Talk                     43   

      b_point_samplereference  \
0                      1074.0   
1                      1849.0   
2                      2518.0   
3                      3252.0   
4                      3933.0   
...                       ...   
4990                  26419.0   
4991                  27042.0   
4992                  27719.0   
4993                  28395.0   
4994                  29097.0   

      arbol2017-isoelectric-crossings_forouzanfar2018  \
0                                              1111.0   
1                                              1844.0   
2                                              2558.0   
3                                              3264.0   
4                                              3942.0   
...                                               ...   
4990                                          26444.0   
4991                                          27051.0   
4992                                          27696.0   
4993                                          28449.0   
4994                                          29132.0   

      arbol2017-isoelectric-crossings_linear-interpolation  \
0                                                1111.0      
1                                                1844.0      
2                                                2558.0      
3                                                3264.0      
4                                                3942.0      
...                                                 ...      
4990                                            26444.0      
4991                                            27051.0      
4992                                            27696.0      
4993                                            28449.0      
4994                                            29132.0      

      arbol2017-isoelectric-crossings_none  \
0                                   1111.0   
1                                   1844.0   
2                                   2558.0   
3                                   3264.0   
4                                   3942.0   
...                                    ...   
4990                               26444.0   
4991                               27051.0   
4992                               27696.0   
4993                               28449.0   
4994                               29132.0   

      arbol2017-second-derivative_forouzanfar2018  \
0                                          1080.0   
1                                          1810.0   
2                                          2494.0   
3                                          3215.0   
4                                          3881.0   
...                                           ...   
4990                                      26385.0   
4991                                      27018.0   
4992                                      27679.0   
4993                                      28408.0   
4994                                      29103.0   

      arbol2017-second-derivative_linear-interpolation  ...  \
0                                               1080.0  ...   
1                                               1809.0  ...   
2                                               2494.0  ...   
3                          

### Save the converted dataframe

In [21]:
if save_results:
    df_empkins_pivot_ms.to_csv(result_path.joinpath("pivot_dataframe_b_point/empkins_pivot_dataframe_b_point_all_algos_ms.csv"))